<a href="https://colab.research.google.com/github/Buse-cetin/Topic_Modelling/blob/main/Bertopicc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

unwanted_cleanda zaten temizlenmiş veriş veriler olduğu için burdaki temizleme kodları silinebilir.

In [ ]:
datafile = '/content/drive/MyDrive/unwanted_clean.csv'

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np
query_df = pd.read_csv(datafile,error_bad_lines=False)
query_df = query_df.iloc[:8000]
query_df['tweet'] = query_df['tweet'].astype(str)
query_df['tweet'].head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


0    dakikadan fazladır taksi dksı görünen otobüsü ...
1    olimpiyat cadde üzeri pendik buraya araç park ...
2    parklarda kontrol bakımları aksatarak canlarla...
3    parklarda kontrol bakımları aksatarak canlarla...
4    bugün bir taksici terörüne tanık oldumtaksici ...
Name: tweet, dtype: object

In [ ]:
import re
import string
from gensim import corpora, models, similarities 

Tweetlerin emojilerden ve sembollerden temizlenmesi

In [ ]:
import re

def deEmoji(text):

  emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emojiler
          u"\U0001F300-\U0001F5FF"  # ifadeler
          u"\U0001F680-\U0001F6FF"  # semboller
          u"\U0001F1E0-\U0001F1FF"  
                            "]+", flags=re.UNICODE)
  return str(emoji_pattern.sub('', text) )

Stopwords 

In [ ]:
with open('/content/drive/MyDrive/stopwords-tr (1).txt', 'r') as f:
    myList = [line.strip() for line in f]

Tokenization

In [ ]:
def word_tokenize(sentence):
      
    
    acronym_each_dot = r"(?:[a-zğçşöüı]\.){2,}"
    acronym_end_dot = r"\b[a-zğçşöüı]{2,3}\."
    suffixes = r"[a-zğçşöüı]{3,}' ?[a-zğçşöüı]{0,3}"
    numbers = r"\d+[.,:\d]+"
    any_word = r"[a-zğçşöüı]+"
    punctuations = r"[a-zğçşöüı]*[.,!?;:]"
    word_regex = "|".join([acronym_each_dot,
                           acronym_end_dot,
                           suffixes,
                           numbers,
                           any_word,
                           punctuations])
    
    sentence = re.compile("%s"%word_regex, re.I).findall(sentence)
    return sentence

Büyük-küçük harf dönüşümü

In [ ]:
def initial_clean(text):
     
     text = text.translate(str.maketrans('', '', string.punctuation))
     text = text.lower() 
     text = word_tokenize(text)
     return text

In [ ]:
def remove_stop_words(text):
     stop_words = myList
     new_stopwords = ["pkk","imamoğlu","akp","chp","terör","izmir","iki","ibb","ibbsikayet","istanbul","olimpiyat","belediye","blk","den","soylu","belediye"]
     stop_words.extend(new_stopwords)
     return [word for word in text if word not in stop_words]

İstenmeyen kelimelerin çıkartılması

In [ ]:
unwanted_data= [ "pkk","imamoğlu","akp","chp","terör","izmir","iki","ibb","ibbsikayet","istanbul","nin","belediye","blk","olimpiyat","soylu","belediye"] 
query_df = pd.DataFrame(query_df, columns = ["tweet"])
unwanted= query_df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in unwanted_data))

print(unwanted)

0       dakikadan fazladır taksi dksı görünen otobüsü ...
1       cadde üzeri pendik buraya araç park etmek yasa...
2       parklarda kontrol bakımları aksatarak canlarla...
3       parklarda kontrol bakımları aksatarak canlarla...
4       bugün bir taksici terörüne tanık oldumtaksici ...
                              ...                        
7995    bir mesai çıkışlarında yol çilesi olmasaydı iy...
7996                               manifestliyo anlamadim
7997    asgariücret zam yansımaları erken başladı topl...
7998    vesile yi kınıyor ekonomi yönetimleri başarısı...
7999    kes tatavayi ibo peynirin kilosu kiymanin kilo...
Name: tweet, Length: 8000, dtype: object


Yukarıdaki tüm fonksiyonları tek bir işleve toplar

In [ ]:
def apply_all(text):
     
     return remove_stop_words(initial_clean(deEmoji(text)))

Token İşlemi

In [ ]:
import time   
t1 = time.time()   
query_df['tokenized_texts'] = query_df['tweet'].apply(apply_all) 
t2 = time.time()  
print("Time to clean and tokenize", len(query_df), "texts:", (t2-t1)/60, "min") 

Time to clean and tokenize 8000 texts: 1.4863162994384767 min


In [ ]:
query_df.head(6)

,tweet,tokenized_texts
0,dakikadan fazladır taksi dksı görünen otobüsü ...,"[dakikadan, fazladır, taksi, dksı, görünen, ot..."
1,olimpiyat cadde üzeri pendik buraya araç park ...,"[cadde, üzeri, pendik, buraya, araç, park, etm..."
2,parklarda kontrol bakımları aksatarak canlarla...,"[parklarda, kontrol, bakımları, aksatarak, can..."
3,parklarda kontrol bakımları aksatarak canlarla...,"[parklarda, kontrol, bakımları, aksatarak, can..."
4,bugün bir taksici terörüne tanık oldumtaksici ...,"[bugün, taksici, terörüne, tanık, oldumtaksici..."
5,akşam otobüse binemedikkendinizle gurur duyun ...,"[akşam, otobüse, binemedikkendinizle, gurur, d..."


BerTopic

In [ ]:
!pip install bertopic[visualization] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 68.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
from copy import deepcopy
from bertopic import BERTopic

In [ ]:
docs = list(query_df.loc[:, "tweet"].values)

In [ ]:
docs[:5]

['dakikadan fazladır taksi dksı görünen otobüsü bekliyoruz taksi reddetti istanbulun ulaşım sıkıntısından gerçekten yıldım taksi ibbşikayet',
 'olimpiyat cadde üzeri pendik buraya araç park etmek yasak arkadaslar usulsüz şekilde park ediyorotobüs ler araç lar zaman zaman zorlanıyorgeçmekte ibbşikayet emniyet trafikşube',
 'parklarda kontrol bakımları aksatarak canlarla oynuyorsunuz kadikoybelediyesi kadikoysikayet',
 'parklarda kontrol bakımları aksatarak canlarla oynuyorsunuz kadikoybelediyesi kadikoysikayet',
 'bugün bir taksici terörüne tanık oldumtaksici bagdat cadkarşıya geçmekte bir çifti kalsın eziyorduadam dikkat etmesini söyleyince küfrettikavga çıkarttı adamın gözüne sprey gaz sıkıp kaçtıbu taksici tehlikelitjc ibbşikayet taksiciterörü']

In [ ]:
model = BERTopic(language="turkish")

In [ ]:
topics, probs = model.fit_transform(docs)

In [ ]:
model.get_topic_freq()

,Topic,Count
0,-1,3723
1,0,363
2,1,291
3,2,240
4,3,177
...,...,...
106,105,11
107,106,11
108,107,11
109,108,11


In [ ]:
model.get_topic(2)

[('türkiye', 0.02928423608067783),
 ('türk', 0.029193043133881005),
 ('istanbul', 0.02462448643489653),
 ('istanbulda', 0.023581899123863544),
 ('toplu', 0.010970715607554892),
 ('ulaşım', 0.010295615262336974),
 ('zam', 0.009350441618791519),
 ('büyükşehir', 0.008898325849784868),
 ('türkiyeyi', 0.008206557071207807),
 ('türkçe', 0.007448583665047516)]

In [ ]:
model.get_topic(4)

[('otobüs', 0.0714009973228333),
 ('otobüsü', 0.04075103558797718),
 ('otobüsler', 0.028688532266003493),
 ('otobüsleri', 0.023702969592990057),
 ('iett', 0.02055689219713246),
 ('otobüse', 0.015045537161375008),
 ('saat', 0.014909171147908611),
 ('durakta', 0.012888511902611088),
 ('geç', 0.012683305549045388),
 ('st', 0.012649172531846544)]

In [ ]:
model.get_topic(5)

[('egm', 0.04003426751450194),
 ('aldım', 0.026269225565933998),
 ('şeysi', 0.0258482873425478),
 ('yaptınız', 0.025317978804257207),
 ('manifest', 0.02497186952937267),
 ('hay', 0.023350701833188173),
 ('anlatsana', 0.023350701833188173),
 ('kaldır', 0.02220160655707646),
 ('akbil', 0.021956514244152143),
 ('aga', 0.021311178840836168)]

In [ ]:
model.get_topic(10)

[('trafik', 0.056987606007715366),
 ('yaya', 0.027257048576956017),
 ('trafiği', 0.0259708046759922),
 ('yol', 0.024514680058418448),
 ('park', 0.02391474155620198),
 ('trafiğe', 0.02217529549676279),
 ('kadıköy', 0.019733642447152217),
 ('scooter', 0.019134027067324963),
 ('maltepe', 0.018746738386720727),
 ('araç', 0.018041375484189173)]

BerTopic Görselleştirme

In [ ]:
model.visualize_topics()